In [558]:
def clean(text):
    text =  text.split('(')[0].split('\xa0')[0].replace('$', '').replace('&','').replace('#', '')\
        .replace('* ','').replace('*#', '').replace('*','').replace('/', ', ')
    if text[-1] == ' ':
        text = text[:-1]
    return text
    

import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

url = 'https://www.ncaa.com'
soup = bs(requests.get(url).text, 'lxml')

urls = [url+tag['href'] for tag in soup.find_all('a', {'href':re.compile('/sports/')})]

sports = []
seasons = []
champions = []

for u in urls:
    
    url = 'https://www.ncaa.com'+bs(requests.get(u).text, 'lxml').find('a', {'href':re.compile('/history')})['href']
    soup = bs(requests.get(url).text, 'lxml')
    
    sport = soup.find('h2', {'class':'page-title'}).text.strip()
    table = soup.find_all('table')[0]
    n = len(table.find_all('th'))

    ssns = [int(tag.text.strip()) for tag in table.find_all('td')[0::n]]
    champs = [clean(tag.text.strip()) for tag in table.find_all('td')[1::n]]
    
    sports += [sport for i in range(len(champs))]
    seasons += ssns
    champions += champs

df = pd.DataFrame({'Sport':sports, 'Season':seasons, 'Champion':champions})

df

,Sport,Season,Champion
0,Men's Cross Country,2019,BYU
1,Men's Cross Country,2018,Northern Arizona
2,Men's Cross Country,2017,Northern Arizona
3,Men's Cross Country,2016,Northern Arizona
4,Men's Cross Country,2015,Syracuse
5,Men's Cross Country,2014,Colorado
6,Men's Cross Country,2013,Colorado
7,Men's Cross Country,2012,Oklahoma State
8,Men's Cross Country,2011,Wisconsin
9,Men's Cross Country,2010,Oklahoma State


In [560]:
piv = df[df.Season >= 1973].pivot_table(values='Season', index=['Champion'], columns='Sport', aggfunc='count').fillna(0).astype(int)
droppers = []
for idx in piv.index:
    if ', ' in idx:
        for team in idx.split(', '):
            if team in piv.index:
                piv.loc[team] += piv.loc[idx]
            else:
                piv.loc[team] = piv.loc[idx]
        droppers.append(idx)

piv = piv.drop(droppers)

piv["Men's CC/Track"] = piv["Men's Cross Country"] + piv["Men's Indoor Track & Field"] + piv["Men's Outdoor Track & Field"]
piv["Women's CC/Track"] = piv["Women's Cross Country"] + piv["Women's Indoor Track & Field"] + piv["Women's Outdoor Track & Field"]

piv = piv.drop(["Men's Cross Country", "Men's Indoor Track & Field", "Men's Outdoor Track & Field", "Women's Cross Country", "Women's Indoor Track & Field", "Women's Outdoor Track & Field"], axis=1)
piv = piv.sort_index(axis=1)


def weight(row):
    return row["Baseball"]*0.025 + row["Beach Volleyball"]*0.0021 + row["Bowling"]*0.0042 + row["Fencing"]*0.0108 + \
        row["Field Hockey"]*0.0141 + row["Football"]*0.3863 + row["Men\'s Basketball"]*0.1457 + \
        row["Men\'s CC/Track"]*0.0107 + row["Men\'s Golf"]*0.0072 + row["Men\'s Gymnastics"]*0.0049 + \
        row["Men\'s Ice Hockey"]*0.0642 + row["Men\'s Lacrosse"]*0.0295 + row["Men\'s Soccer"]*0.0148 + \
        row["Men\'s Swimming"]*0.0098 + row["Men\'s Tennis"]*0.0068 + row["Men\'s Volleyball"]*0.0115 + \
        row["Men\'s Water Polo"]*0.0115 + row["Rifle"]*0.0016 + row["Rowing"]*0.0156 + row["Skiing"]*0.0203 + \
        row["Softball"]*0.0143 + row["Women\'s Basketball"]*0.0345 + row["Women\'s CC/Track"]*0.0134 + \
        row["Women\'s Golf"]*0.0069 + row["Women\'s Gymnastics"]*0.0173 + row["Women\'s Ice Hockey"]*0.0283 + \
        row["Women\'s Lacrosse"]*0.0159 + row["Women\'s Soccer"]*0.0148 + row["Women\'s Swimming"]*0.0109 + \
        row["Women\'s Tennis"]*0.0074 + row["Women\'s Volleyball"]*0.0148 + row["Women\'s Water Polo"]*0.0094 + \
        row["Wrestling"]*0.0155


piv['Weighted Total'] = [weight(piv.loc[idx]) for idx in piv.index]
piv = piv.sort_values(by='Weighted Total', ascending=False)
piv

Sport,Baseball,Beach Volleyball,Bowling,Fencing,Field Hockey,Football,Men's Basketball,Men's CC/Track,Men's Golf,Men's Gymnastics,...,Women's Gymnastics,Women's Ice Hockey,Women's Lacrosse,Women's Soccer,Women's Swimming,Women's Tennis,Women's Volleyball,Women's Water Polo,Wrestling,Weighted Total
Champion,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,0,9,0,0,2,0,...,6,0,0,0,0,0,0,0,0,3.6161
Southern California,4,2,0,0,0,4,0,1,0,0,...,0,0,0,2,1,2,3,5,0,2.1915
Miami,4,0,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0384
Florida,1,0,0,0,0,3,2,9,3,0,...,3,0,0,1,2,7,0,0,0,1.8089
Oklahoma,1,0,0,0,0,4,0,0,2,12,...,4,0,0,0,0,0,0,0,1,1.7853
LSU,6,0,0,0,0,3,0,5,1,0,...,0,0,0,0,0,0,0,0,0,1.7180
UCLA,1,2,0,0,0,0,3,4,2,2,...,7,0,0,1,0,2,4,7,0,1.4684
North Carolina,0,0,0,0,8,0,5,0,0,0,...,0,0,2,21,0,0,0,0,0,1.3955
Notre Dame,0,0,0,6,0,3,0,0,0,0,...,0,0,0,3,0,0,0,0,0,1.3519


In [570]:
piv['Weighted Total']

Champion
Alabama                   3.6161
Southern California       2.1915
Miami                     2.0384
Florida                   1.8089
Oklahoma                  1.7853
LSU                       1.7180
UCLA                      1.4684
North Carolina            1.3955
Notre Dame                1.3519
Nebraska                  1.3199
Stanford                  1.2024
Clemson                   1.1957
Connecticut               1.0624
Texas                     0.9401
Ohio State                0.9237
Duke                      0.8875
Florida State             0.8754
Colorado                  0.8048
Penn St.                  0.7726
Georgia                   0.7616
Tennessee                 0.7417
Michigan                  0.7280
Arkansas                  0.6728
Maryland                  0.6485
Wisconsin                 0.6325
Kentucky                  0.5994
Villanova                 0.5684
Indiana                   0.5653
Minnesota                 0.5445
Virginia                  0.5399
 